In [1]:
from datetime import datetime
import requests
import json

In [4]:
f = open('issues.json')
issues = json.load(f)

2022-08-17T05:13:54Z


In [5]:
f = open('commits.json')
commits = json.load(f)

In [9]:
print(commits[0]["commit"])

{'author': {'name': 'Doomsdayrs', 'email': '38189170+Doomsdayrs@users.noreply.github.com', 'date': '2022-10-03T01:07:57Z'}, 'committer': {'name': 'GitHub', 'email': 'noreply@github.com', 'date': '2022-10-03T01:07:57Z'}, 'message': 'Update update.json', 'tree': {'sha': 'ab1bea32cc167f377e7328e8e9829963cbfcf5a7', 'url': 'https://api.github.com/repos/shosetsuorg/shosetsu/git/trees/ab1bea32cc167f377e7328e8e9829963cbfcf5a7'}, 'url': 'https://api.github.com/repos/shosetsuorg/shosetsu/git/commits/692ab6ea892c6e03aa6d1530f460ceb873a1a029', 'comment_count': 0, 'verification': {'verified': True, 'reason': 'valid', 'signature': '-----BEGIN PGP SIGNATURE-----\n\nwsBcBAABCAAQBQJjOjXtCRBK7hj4Ov3rIwAA/mkIADUa5Wey6d5tWl+ynAh+dOyS\nyh3YvoVEHXAXvA+C4cHGXVIx+SMhj3q8JRapaVSNM5UstmG89R8PqhBsSnvLybVP\nMf5svasFJACQVOOP6hy/6bJmZv3BI4AsySnTQskUi+TfO6gR1/+8gl9hc10Wbtm+\nPrTIypZQ3uV98u0xiSY+XQispfDXNhbFuIOZB8lh0Xc7T07c+MdY02KYrDpkJHeU\n31PQiJ4kjNAPFvuZfGeFWQWw+D5wtPOnrGAoU/w32lbioYFAlSsDNDxtEowot5t3\nzFsjpksr6U+

In [11]:
issue_commit_map = {}

for issue in issues:
    issue_created_at = issue["created_at"]
    issue_closed_at = issue["closed_at"]
    issue_created_at = datetime.fromisoformat(issue_created_at.replace('Z', '+00:00'))
    issue_closed_at = datetime.fromisoformat(issue_closed_at.replace('Z', '+00:00'))
    issue_commit_map[issue["id"]] = {"issue_title": issue["title"], "commit_messages":[]}
    for commit in commits:
    # print("created_at",obj["created_at"], "\nclosed_at", obj["closed_at"])
        commit_made_at = commit["commit"]["committer"]["date"]
        commit_made_at = datetime.fromisoformat(commit_made_at.replace('Z', '+00:00'))
        # print(commit_made_at)
        if commit_made_at>=issue_created_at and commit_made_at<=issue_closed_at:
            # new_item = {"issue_id": issue["id"], "commit_message":commit["commit"]["message"]}
            issue_commit_map[issue["id"]]["commit_messages"].append(commit["commit"]["message"])

# print(issue_commit_map)

issue_commit_map_list = list(issue_commit_map.values())
print(issue_commit_map_list)

[{'issue_title': 'Move app package to app.shosetsu.android', 'commit_messages': ['Move app package to app.shosetsu.android']}, {'issue_title': 'Temp fix for backup restore with external repos', 'commit_messages': ['Temp fix for backup restore with external repos']}, {'issue_title': '[Bug] [App lags] ', 'commit_messages': ['Fix proguard-rules.pro', 'Move app package to app.shosetsu.android', 'Release debug update for r2417', 'Direct disclaimer to shosetsu website', 'Prevent concurrent modifies in CatalogViewModel via sync & copy', 'Add privacy policy button to about', 'Add more logs to RestoreBackupWorker.kt', 'Temp fix for backup restore with external repos']}, {'issue_title': 'Fullscreen Settings for Reader', 'commit_messages': ['Fix padding', 'Fix enabled parameter on switch settings', 'MatchFullscreen is false by default', 'Address comment', 'Match fullscreen to focus option', 'Add fullscreen option', 'Clean up\n\nHandle various build warnings and removing older classes', 'Update de

In [21]:
from openai import OpenAI

v1 = "sk-5GmSxGb0TLx6jzpzhFbhT3B"
v2 = "lbkFJm3OoryQONwklze43ygTB"
OPENAI_API_KEY = v1+v2


client = OpenAI(api_key=OPENAI_API_KEY)


In [46]:
issue_passed_to_OPENAI = issue_commit_map_list[3]["issue_title"]
string2 = issue_commit_map_list[4]["commit_messages"]
string2 = ", ".join(string2)
string2 = string2.split('\n')
string2 = " ".join(string2)
string2 = string2.split('\t')
commit_messages_passed_to_OPENAI = " ".join(string2)

print("issue", issue_passed_to_OPENAI)
print("commtis", commit_messages_passed_to_OPENAI)

prompt_sent_to_OPENAI = "issue: "+ issue_passed_to_OPENAI + "\ncommit: "+ commit_messages_passed_to_OPENAI
print("prompt\n",prompt_sent_to_OPENAI)


issue Fullscreen Settings for Reader
commtis Fix animation stutter, Re-add elevation and remove top content padding, Fix cover ContentScale, Fix expanded cover having a ratio and being rounded, NovelCard fixes, forced center alignment and outside padding, Fix scrolling filters, Add unread badge click, Licenses, Use a standard cover ratio, UI Cleanup and improvements
prompt
 issue: Fullscreen Settings for Reader
commit: Fix animation stutter, Re-add elevation and remove top content padding, Fix cover ContentScale, Fix expanded cover having a ratio and being rounded, NovelCard fixes, forced center alignment and outside padding, Fix scrolling filters, Add unread badge click, Licenses, Use a standard cover ratio, UI Cleanup and improvements


In [49]:
##USE only when you need only percentage in the answer
# command_to_OPENAI = "You will receive two objects. One is an issue from gitHub. The other is a list of commit messages from gitHub. Now you have to give the rating out of 10, based on how closely related the issue and the commit are. Rating also should depend upon how closely they are refering to the same thing. There might be some commits which are not related to the issue, but try to ignore them and see if the commits which can be related are solving the issue or not. Do not give any explanation or any words. Just use a percentage to specify how close they are.."

##USE when you also need an explanation for it.
command_to_OPENAI = "You will receive two objects. One is an issue from gitHub. The other is a list of commit messages from gitHub. Now you have to give the rating out of 10, based on how closely related the issue and the commit are. Rating also should depend upon how closely they are refering to the same thing. There might be some commits which are not related to the issue, but try to ignore them and see if the commits which can be related are solving the issue or not. Also explain why do you think so?"

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": command_to_OPENAI},
    {"role": "user", "content": prompt_sent_to_OPENAI}
  ]
)

In [50]:
print(completion.choices[0].message.content)

70%
